In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.3.1


In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1)  # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1)  # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10)  # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10)  # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [7]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

In [8]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [9]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [10]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_xavier'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [11]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.5693, train_loss: 2.03429079, train_accuracy: 0.3828, test_Accuracy: 0.2607
Epoch: [ 0] [    1/  468] time: 0.7503, train_loss: 1.94159198, train_accuracy: 0.4531, test_Accuracy: 0.3749
Epoch: [ 0] [    2/  468] time: 0.9368, train_loss: 1.78048670, train_accuracy: 0.6172, test_Accuracy: 0.5156
Epoch: [ 0] [    3/  468] time: 1.2307, train_loss: 1.63410199, train_accuracy: 0.6406, test_Accuracy: 0.6261
Epoch: [ 0] [    4/  468] time: 1.4907, train_loss: 1.57172060, train_accuracy: 0.7578, test_Accuracy: 0.6923
Epoch: [ 0] [    5/  468] time: 1.6487, train_loss: 1.30889249, train_accuracy: 0.8359, test_Accuracy: 0.7128
Epoch: [ 0] [    6/  468] time: 1.8117, train_loss: 1.37057662, train_accuracy: 0.6953, test_Accuracy: 0.7003
Epoch: [ 0] [    7/  468] time: 1.9917, train_loss: 1.21696591, train_accuracy: 0.6875, test_Accuracy: 0.7051
Epoch: [ 0] [    8/  468] time: 2.1817,

Epoch: [ 0] [   74/  468] time: 12.5712, train_loss: 0.36500192, train_accuracy: 0.9141, test_Accuracy: 0.9106
Epoch: [ 0] [   75/  468] time: 12.7212, train_loss: 0.27902994, train_accuracy: 0.9062, test_Accuracy: 0.9116
Epoch: [ 0] [   76/  468] time: 12.8993, train_loss: 0.21593533, train_accuracy: 0.9219, test_Accuracy: 0.9113
Epoch: [ 0] [   77/  468] time: 13.0493, train_loss: 0.20784791, train_accuracy: 0.9297, test_Accuracy: 0.9087
Epoch: [ 0] [   78/  468] time: 13.1952, train_loss: 0.38299561, train_accuracy: 0.9141, test_Accuracy: 0.9067
Epoch: [ 0] [   79/  468] time: 13.3372, train_loss: 0.33707798, train_accuracy: 0.9141, test_Accuracy: 0.9094
Epoch: [ 0] [   80/  468] time: 13.4776, train_loss: 0.34600487, train_accuracy: 0.8672, test_Accuracy: 0.9126
Epoch: [ 0] [   81/  468] time: 13.6185, train_loss: 0.29274151, train_accuracy: 0.9219, test_Accuracy: 0.9145
Epoch: [ 0] [   82/  468] time: 13.7605, train_loss: 0.21173650, train_accuracy: 0.9297, test_Accuracy: 0.9119
E

Epoch: [ 0] [  148/  468] time: 24.8922, train_loss: 0.23251842, train_accuracy: 0.9062, test_Accuracy: 0.9356
Epoch: [ 0] [  149/  468] time: 25.0342, train_loss: 0.29167753, train_accuracy: 0.9219, test_Accuracy: 0.9333
Epoch: [ 0] [  150/  468] time: 25.1862, train_loss: 0.14566842, train_accuracy: 0.9531, test_Accuracy: 0.9336
Epoch: [ 0] [  151/  468] time: 25.3242, train_loss: 0.12169126, train_accuracy: 0.9844, test_Accuracy: 0.9330
Epoch: [ 0] [  152/  468] time: 25.4632, train_loss: 0.31648949, train_accuracy: 0.9141, test_Accuracy: 0.9334
Epoch: [ 0] [  153/  468] time: 25.6138, train_loss: 0.21942604, train_accuracy: 0.9375, test_Accuracy: 0.9343
Epoch: [ 0] [  154/  468] time: 25.7628, train_loss: 0.22286320, train_accuracy: 0.9219, test_Accuracy: 0.9352
Epoch: [ 0] [  155/  468] time: 25.9738, train_loss: 0.20517868, train_accuracy: 0.9609, test_Accuracy: 0.9368
Epoch: [ 0] [  156/  468] time: 26.1487, train_loss: 0.21485981, train_accuracy: 0.9531, test_Accuracy: 0.9368
E

Epoch: [ 0] [  222/  468] time: 37.2058, train_loss: 0.16426176, train_accuracy: 0.9375, test_Accuracy: 0.9466
Epoch: [ 0] [  223/  468] time: 37.4288, train_loss: 0.23778249, train_accuracy: 0.9219, test_Accuracy: 0.9457
Epoch: [ 0] [  224/  468] time: 37.6508, train_loss: 0.15885505, train_accuracy: 0.9609, test_Accuracy: 0.9440
Epoch: [ 0] [  225/  468] time: 37.8198, train_loss: 0.19319999, train_accuracy: 0.9297, test_Accuracy: 0.9438
Epoch: [ 0] [  226/  468] time: 37.9638, train_loss: 0.17174534, train_accuracy: 0.9453, test_Accuracy: 0.9452
Epoch: [ 0] [  227/  468] time: 38.1235, train_loss: 0.20430212, train_accuracy: 0.9453, test_Accuracy: 0.9467
Epoch: [ 0] [  228/  468] time: 38.2785, train_loss: 0.20403877, train_accuracy: 0.9297, test_Accuracy: 0.9483
Epoch: [ 0] [  229/  468] time: 38.4225, train_loss: 0.15151195, train_accuracy: 0.9531, test_Accuracy: 0.9493
Epoch: [ 0] [  230/  468] time: 38.5736, train_loss: 0.17571326, train_accuracy: 0.9297, test_Accuracy: 0.9486
E

Epoch: [ 0] [  296/  468] time: 49.7918, train_loss: 0.11900306, train_accuracy: 0.9531, test_Accuracy: 0.9500
Epoch: [ 0] [  297/  468] time: 49.9518, train_loss: 0.18582390, train_accuracy: 0.9297, test_Accuracy: 0.9505
Epoch: [ 0] [  298/  468] time: 50.1838, train_loss: 0.10960028, train_accuracy: 0.9375, test_Accuracy: 0.9531
Epoch: [ 0] [  299/  468] time: 50.3568, train_loss: 0.09541735, train_accuracy: 0.9766, test_Accuracy: 0.9537
Epoch: [ 0] [  300/  468] time: 50.5388, train_loss: 0.17788172, train_accuracy: 0.9609, test_Accuracy: 0.9541
Epoch: [ 0] [  301/  468] time: 50.6938, train_loss: 0.09658676, train_accuracy: 0.9766, test_Accuracy: 0.9535
Epoch: [ 0] [  302/  468] time: 50.8488, train_loss: 0.15980491, train_accuracy: 0.9766, test_Accuracy: 0.9521
Epoch: [ 0] [  303/  468] time: 51.0078, train_loss: 0.16951530, train_accuracy: 0.9609, test_Accuracy: 0.9508
Epoch: [ 0] [  304/  468] time: 51.2478, train_loss: 0.19613127, train_accuracy: 0.8906, test_Accuracy: 0.9520
E

Epoch: [ 0] [  370/  468] time: 63.4502, train_loss: 0.10262080, train_accuracy: 0.9766, test_Accuracy: 0.9577
Epoch: [ 0] [  371/  468] time: 63.5982, train_loss: 0.10243070, train_accuracy: 0.9922, test_Accuracy: 0.9569
Epoch: [ 0] [  372/  468] time: 63.7492, train_loss: 0.09034268, train_accuracy: 0.9688, test_Accuracy: 0.9564
Epoch: [ 0] [  373/  468] time: 63.9739, train_loss: 0.06468111, train_accuracy: 0.9844, test_Accuracy: 0.9556
Epoch: [ 0] [  374/  468] time: 64.1488, train_loss: 0.17493816, train_accuracy: 0.9375, test_Accuracy: 0.9553
Epoch: [ 0] [  375/  468] time: 64.3088, train_loss: 0.19505160, train_accuracy: 0.9219, test_Accuracy: 0.9568
Epoch: [ 0] [  376/  468] time: 64.5478, train_loss: 0.16347274, train_accuracy: 0.9688, test_Accuracy: 0.9588
Epoch: [ 0] [  377/  468] time: 64.7165, train_loss: 0.16701551, train_accuracy: 0.9453, test_Accuracy: 0.9611
Epoch: [ 0] [  378/  468] time: 64.8685, train_loss: 0.16393211, train_accuracy: 0.9375, test_Accuracy: 0.9603
E

Epoch: [ 0] [  445/  468] time: 77.3696, train_loss: 0.12607342, train_accuracy: 0.9609, test_Accuracy: 0.9617
Epoch: [ 0] [  446/  468] time: 77.5496, train_loss: 0.11818455, train_accuracy: 0.9688, test_Accuracy: 0.9628
Epoch: [ 0] [  447/  468] time: 77.6947, train_loss: 0.16778854, train_accuracy: 0.9375, test_Accuracy: 0.9632
Epoch: [ 0] [  448/  468] time: 77.8357, train_loss: 0.03789456, train_accuracy: 0.9922, test_Accuracy: 0.9639
Epoch: [ 0] [  449/  468] time: 78.0487, train_loss: 0.10012229, train_accuracy: 0.9531, test_Accuracy: 0.9642
Epoch: [ 0] [  450/  468] time: 78.2417, train_loss: 0.08859760, train_accuracy: 0.9766, test_Accuracy: 0.9639
Epoch: [ 0] [  451/  468] time: 78.4027, train_loss: 0.12084725, train_accuracy: 0.9531, test_Accuracy: 0.9635
Epoch: [ 0] [  452/  468] time: 78.5617, train_loss: 0.12477472, train_accuracy: 0.9531, test_Accuracy: 0.9610
Epoch: [ 0] [  453/  468] time: 78.7827, train_loss: 0.15257056, train_accuracy: 0.9609, test_Accuracy: 0.9614
E